#  **Can mental health professionals trust AI-based systems to prevent suicide? Effects of educational intervention and explanations on Trust**

**Developer:** Adonias Caetano de Oliveira

**Version:** Interface Without XAI

## **Library installation and environment configuration**

In [ ]:
from google.colab import drive

PATH = '/content/drive'
drive.mount(PATH)

Mounted at /content/drive


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
!pip install Unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

## **Importing library**

In [ ]:
# Auxiliaries
import pandas as pd
import random
import time
import datetime
import numpy as np
import io
from scipy.special import expit
import re

# Deep learning and BERT
import torch
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from tqdm.notebook import tqdm

#NLP
from unidecode import unidecode
from string import punctuation

# Interface with Gradio
import gradio as gr

## **Reading the data set**

In [ ]:
url = '<link google drive of dataset CSV'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
dataset = pd.read_csv(read_url)

# display the first 5 rows
dataset.head()

,text,target
0,Aquela vontade de acabar com a minha vida voltou,1
1,to triste e com vontade de acabar com a minha ...,1
2,Corinthians ta querendo acabar com minha vida ...,0
3,Alguém poderia por favor me dar um tiro a acab...,1
4,TAYLOR TU VAI acabar com a minha vida MULHER,0


In [ ]:
dataset.shape

(3788, 2)

## **Text pre-processing**

In [ ]:
def clean(sentences):

  new_texts = []

  for text in sentences:
    text = text.lower()
    text = re.sub('@[^\s]+', '', text)
    text = unidecode(text)
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
    text = ''.join(c for c in text if c not in punctuation)
    new_texts.append(text)

  return new_texts

In [ ]:
dataset['text'] = clean(dataset['text'].values)

In [ ]:
def get_examples_sent():
  negativos = dataset.loc[dataset['target'] == 0].sample(n = 3)
  positivos = dataset.loc[dataset['target'] == 1].sample(n = 3)

  return list(negativos['text'].values) + list(positivos['text'].values)

## **Classification with BERT**

In [ ]:
PRETRAINED_LM = 'neuralmind/bert-large-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

BertTokenizer(name_or_path='neuralmind/bert-large-portuguese-cased', vocab_size=29794, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
def createDataloader(text):
  test_input_ids, test_att_masks = encode([text])
  BATCH_SIZE = 16
  test_y = torch.LongTensor([0])
  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
  test_sampler = SequentialSampler(test_dataset)
  test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)
  return test_dataloader

In [ ]:
labels_names = ['negativo', 'positivo']

In [ ]:
device = torch.device("cpu")

In [ ]:
MODEL_PATH = PATH + '/My Drive/Colab Notebooks/Interface XAI - BERT - LIME/Models/model_95.bin'
def get_bert_model():
    N_labels = 2
    model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

    model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu')), strict=False)

    return model

model = get_bert_model()

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **Classificação de Texto**

In [ ]:
def predict(text):
  model.eval()
  test_dataloader = createDataloader(text)


  with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        logits = output.logits.cpu().detach().numpy()
        index_pred = np.argmax(logits,axis=-1)[0]
        probabilities = expit(logits)[0]


  df = pd.DataFrame(columns=['Label', 'Probabilidade'])
  df['Label'] = labels_names
  df['Probabilidade'] = probabilities

  return index_pred, labels_names[index_pred], df

**Verifica se o texto é 100% negativo**

In [ ]:
# Palavras que podem indicm pensamentos suicidas
suicidal_word_list = ["suicida",  "suicídio", "sucidio", "matar", "bilhete", "carta",
             "despedida", "adeus", "acabar", "vida", "nunca", "acordar",
             "acorda", "não", "nao", "consigo", "continuar", "vale", "pena",
             "viver", "pronto", "para", "pular", "dormir", "sempre", "quero",
             "morrer", "morto", "melhor", "sem", "mim", "plano", "pacto",
             "cansado", "sozinho", "dormi", "sonho", "feliz", "só", "melhores",
             "ajuda", "valor", "entende", "bom", "escuridão", "ama", "amam", "confio", "passa", "passar", "vou", "outra"]

In [ ]:
def words_in_string(word_list, a_string):
    return set(word_list).intersection(a_string.split())

def is_negative(text):
  return not text or not text.strip() or not words_in_string(suicidal_word_list, text)

**Classsifica um texto**

In [ ]:
def classify(text):

  if is_negative(text):
    info = f"<h4>O sistema verificou que o texto digitado não apresenta termos que podem indicar pensamento suicida!<br/>"
    info += f"Portanto, o sistema considerou o texto como 100% Negativo para ideação suicida.<br/>"
    info += f"Não foi aplicado inteligência artificial neste caso.</h4>"
    result = {"Negativo": 1, "Positivo" : 0}
  else:
    # Classificação do texto
    index_pred, label, df = predict(text)
    d = df['Probabilidade'].to_dict()
    result = {"Negativo" if k == 0 else "Positivo" : v for k,v in d.items()}

    prob = df['Probabilidade'].max()

    info = f"<h4>O sistema classificou o texto como {labels_names[index_pred]} para ideação suicida com probabilidade = {prob*100:.2f}%.</h4>"

  return result, info

###**Interface Without XAI**

In [ ]:
with gr.Blocks() as demo_no_xai:
  gr.Markdown(
    """
    # Interface de reconhecimento de ideação suicida sem explicação dos resultados
    Comece a digitar abaixo para ver o resultado.
    """)
  input = gr.Textbox(label="Entrada:", placeholder="Digite ou selecione uma frase positiva ou negativa para ideação suicida...")

  gr.Examples(
        examples = get_examples_sent(),
        inputs = input
  )


  #Botão de classificação
  classificar_btn = gr.Button("Classificar")
  output = gr.Label(label="Predicted Suicidal Ideation")
  info_classification = gr.Markdown("Acione o botão classificar")
  classificar_btn.click(fn=classify, inputs=input, outputs=[output, info_classification])

demo_no_xai.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d9a4f27b66b83dc6bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
